In [ ]:

# Directory containing the shapefiles
shapefile_directory = "ECS_lines_CA/"

# Assuming 'CA_event_data' DataFrame contains event and zone information
CA_event_data['Zone_numeric'] = CA_event_data['zone'].str[:-1]

# Get a list of all shapefile paths in the directory
shapefile_paths = [os.path.join(shapefile_directory, file) for file in os.listdir(shapefile_directory) if file.endswith('.shp')]

for i, shapefile_path in enumerate(shapefile_paths):
    parts = shapefile_path.rsplit('/', 1)
    partsb = parts[1]
    partsb = str(partsb)
    split_parts = partsb.split('_ECS.shp')
    eventname = split_parts[0]
    matching_rows = CA_event_data[CA_event_data['event'] == eventname]
    target_utm_zone = matching_rows['Zone_numeric'].to_numpy()
    target_utm_zone = int(target_utm_zone[0])

    plt.figure()
    gdf = gpd.read_file(shapefile_path)
    gdf = gdf.set_crs('EPSG:4326')
    utm_crs = f"+proj=utm +zone={target_utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    gdf_utm = gdf.to_crs(utm_crs)
    
    # Perform the coordinate transformation

    source_crs = 'EPSG:4326'
    transformer = pyproj.Transformer.from_crs(source_crs, utm_crs, always_xy=True)
    utm_coords = []
    lat_lon_points = [(lat, lon) for lat, lon in zip(ystress, xstress)]

    xstress_utm = []
    ystress_utm = []
    for lat, lon in lat_lon_points:
        utm_x, utm_y = transformer.transform(lon, lat)
        xstress_utm.append(utm_x)
        ystress_utm.append(utm_y)

    # Extract the x and y coordinates from the 'geometry' column
    x_coords = []
    y_coords = []

    counter = 0
    step = 3 # for downsampling

    for line in gdf_utm['geometry']:
        for point in line.coords:
            x, y = point
            if counter % step == 0:
                x_coords.append(x)
                y_coords.append(y)
            counter += 1

    angles = []
    for i in range(1,len(x_coords)-1):
        p0 = [x_coords[i-1], y_coords[i-1]]
        p1 = [x_coords[i], y_coords[i]]
        p2 = [x_coords[i+1], y_coords[i+1]]
        angles.append(ang([p0,p1], [p1,p2]))

    # Calculate the angle of each line segment
    vmin = 0
    vmax = 6
    # Create a scatter plot of the extracted coordinates with colors based on angles in degrees
    x_range = [min(x_coords) - 10000, max(x_coords) + 10000]
    y_range = [min(y_coords) - 10000, max(y_coords) + 10000]

    # Initialize lists to store the cropped data
    cropped_xstress_utm = []
    cropped_ystress_utm = []
    cropped_SHmax = []

    # Crop xstress_utm, ystress_utm, and SHmax to the specified range while keeping the same size
    for x, y, shmax in zip(xstress_utm, ystress_utm, SHmax):
        if x_range[0] <= x <= x_range[1] and y_range[0] <= y <= y_range[1]:
            cropped_xstress_utm.append(x)
            cropped_ystress_utm.append(y)
            cropped_SHmax.append(shmax)

    vmin = -90
    vmax = 90
    plt.scatter(cropped_xstress_utm, cropped_ystress_utm, c=cropped_SHmax, cmap='coolwarm', marker='s', vmin=vmin, vmax=vmax)
    plt.colorbar(label='SHmax orientation at 5 km depth')
    plt.scatter(x_coords[1:-1], y_coords[1:-1], c=angles, cmap='spring', marker='o',  edgecolors='none')
    plt.title(eventname)
    plt.axis('equal')
    plt.colorbar(label='Angle difference (degrees)')
    plt.show()


In [ ]:
angles_segments_allevents = []
var = []
# Directory containing the shapefiles
shapefile_directory = "ECS_lines_CA/"

# Assuming 'CA_event_data' DataFrame contains event and zone information
CA_event_data['Zone_numeric'] = CA_event_data['zone'].str[:-1]

# Get a list of all shapefile paths in the directory
shapefile_paths = [os.path.join(shapefile_directory, file) for file in os.listdir(shapefile_directory) if file.endswith('.shp')]

for i, shapefile_path in enumerate(shapefile_paths):
    parts = shapefile_path.rsplit('/', 1)
    partsb = parts[1]
    partsb = str(partsb)
    split_parts = partsb.split('_ECS.shp')
    eventname = split_parts[0]
    matching_rows = CA_event_data[CA_event_data['event'] == eventname]
    target_utm_zone = matching_rows['Zone_numeric'].to_numpy()
    target_utm_zone = int(target_utm_zone[0])

    plt.figure(dpi=300)
    gdf = gpd.read_file(shapefile_path)
    gdf = gdf.set_crs('EPSG:4326')
    utm_crs = f"+proj=utm +zone={target_utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    gdf_utm = gdf.to_crs(utm_crs)
    
    # Perform the coordinate transformation

    source_crs = 'EPSG:4326'
    transformer = pyproj.Transformer.from_crs(source_crs, utm_crs, always_xy=True)
    utm_coords = []
    lat_lon_points = [(lat, lon) for lat, lon in zip(ystress, xstress)]

    xstress_utm = []
    ystress_utm = []
    for lat, lon in lat_lon_points:
        utm_x, utm_y = transformer.transform(lon, lat)
        xstress_utm.append(utm_x)
        ystress_utm.append(utm_y)

    # Extract the x and y coordinates from the 'geometry' column
    x_coords = []
    y_coords = []

    counter = 0
    step = 3 # for downsampling

    for line in gdf_utm['geometry']:
        for point in line.coords:
            x, y = point
            if counter % step == 0:
                x_coords.append(x)
                y_coords.append(y)
            counter += 1

    angles_segment = []
    for i in range(1,len(x_coords)-1):
        p0 = [x_coords[i-1], y_coords[i-1]]
        p1 = [x_coords[i], y_coords[i]]
        p2 = [x_coords[i+1], y_coords[i+1]]
        angles_segment.append(ang([p0,p1], [p1,p2]))
        angles_segments_allevents.append(ang([p0,p1], [p1,p2]))
    # Calculate the angle of each line segment
    vmin = 0
    vmax = 6
    # Create a scatter plot of the extracted coordinates with colors based on angles in degrees
    x_range = [min(x_coords) - 10000, max(x_coords) + 10000]
    y_range = [min(y_coords) - 10000, max(y_coords) + 10000]

    # Initialize lists to store the cropped data
    cropped_xstress_utm = []
    cropped_ystress_utm = []
    cropped_SHmax = []

    # Crop xstress_utm, ystress_utm, and SHmax to the specified range while keeping the same size
    for x, y, shmax in zip(xstress_utm, ystress_utm, SHmax):
        if x_range[0] <= x <= x_range[1] and y_range[0] <= y <= y_range[1]:
            cropped_xstress_utm.append(x)
            cropped_ystress_utm.append(y)
            cropped_SHmax.append(shmax)

    unit_vectors = []

    # Convert each angle to its corresponding unit vector
    for SHmax_degrees in cropped_SHmax:
        # Convert the angle to radians
        SHmax_radians = np.radians(SHmax_degrees-90)

        # Calculate the unit vector in 2D space
        x = np.cos(SHmax_radians)
        y = np.sin(SHmax_radians)
        z = 0  # Assuming 2D

        # Normalize the unit vector
        magnitude = np.sqrt(x**2 + y**2 + z**2)
        unit_vector = np.array([x / magnitude, y / magnitude, z / magnitude])
 
        unit_vectors.append(unit_vector)
    # Set the desired magnitude
    desired_magnitude = 2000

    # Convert unit vectors to vectors with the desired magnitude
    vectors_with_desired_magnitude = [vector * desired_magnitude for vector in unit_vectors]

    u = [vector[0] for vector in vectors_with_desired_magnitude]  # x-components of unit vectors
    v = [vector[1] for vector in vectors_with_desired_magnitude]  # y-components of unit vectors
    var.append(np.var(angles_segment))
    plt.quiver(cropped_xstress_utm, cropped_ystress_utm, u,v, angles='xy', scale_units='xy', scale=1, color='black', label='SHmax Orientation')
    plt.scatter(x_coords[1:-1], y_coords[1:-1], c=angles_segment, cmap='spring', marker='o',  edgecolors='none')
    plt.title(eventname)
    plt.axis('equal')
    plt.colorbar(label='Angle difference (degrees)')
    plt.show()